In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/recipe

Mounted at /content/drive
/content/drive/MyDrive/recipe


In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import pickle
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re

# Load model and embeddings from pickle file
def load_model_and_embeddings(file_path='model_embeddings.pkl'):
    with open(file_path, 'rb') as f:
        model, embeddings = pickle.load(f)
    print(f"Model and embeddings loaded from {file_path}")
    return model, embeddings

# Function to recommend recipe based on user input
def recommend_recipe(user_input, model, data_embeddings, tokenizer, file_path='Recipe_Info.txt'):
    # Define get_bert_embeddings function inside recommend_recipe
    def get_bert_embeddings(text_list, tokenizer, model):
        inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

    data = pd.read_csv(file_path, delimiter='\t', header=None, names=["combined_features"])
    user_embedding = get_bert_embeddings([user_input], tokenizer, model)
    sim_scores = cosine_similarity(user_embedding, data_embeddings).flatten()
    sim_indices = sim_scores.argsort()[-5:][::-1]
    recommendations = [(data.iloc[i]['combined_features'], sim_scores[i]) for i in sim_indices]
    return recommendations

# Function to extract detailed ingredients from the given text
def extract_ingredients(text):
    match = re.search(r"요리 상세 재료: ([^,]+(?:, [^:]+)+)", text)
    if match:
        ingredients_str = match.group(1).strip()
        ingredients_list = [ingredient.strip() for ingredient in ingredients_str.split(',')]
        return ingredients_list
    else:
        return []

# Function to search for ingredient on Coupang
def search_ingredient_on_coupang(ingredient):
    encoded_keyword = urllib.parse.quote(ingredient)

    # 검색 결과 페이지 URL
    main_url = f"https://www.coupang.com/np/search?component=&q={encoded_keyword}"

    # 헤더 설정
    header = {
        'Host': 'www.coupang.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3',
    }

    try:
        # 검색 결과 페이지 요청
        response = requests.get(main_url, headers=header)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # 첫 번째 상품 링크 가져오기
        link = soup.select_one("a.search-product-link")
        if not link:
            raise Exception("검색 결과가 없습니다.")

        # 상품 페이지 URL
        sub_url = "https://www.coupang.com" + link.attrs['href']

        # 상품 페이지 요청
        response = requests.get(sub_url, headers=header)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # 상품명
        product_name = soup.select_one("h2.prod-buy-header__title").text.strip()

        # 가격
        product_price = soup.select_one("span.total-price > strong").text.strip()

        # 결과를 딕셔너리로 반환
        return {
            "ingredient": ingredient,
            "url": sub_url,
            "product_name": product_name,
            "product_price": product_price
        }

    except requests.exceptions.RequestException as e:
        return {
            "ingredient": ingredient,
            "error": f"HTTP 요청 중 오류 발생: {e}"
        }
    except Exception as e:
        return {
            "ingredient": ingredient,
            "error": f"오류 발생: {e}"
        }

# Function to process recommendations and search for ingredients
def process_recommendations(recommendations):
    if recommendations:
        first_recommendation = recommendations[0][0]
        detailed_text = first_recommendation

        # Extract ingredients from the detailed text
        ingredients_list = extract_ingredients(detailed_text)

        # Search for each ingredient in Coupang
        results = [search_ingredient_on_coupang(ingredient) for ingredient in ingredients_list[:-1]]

        # Print the results
        for result in results:
            if "error" in result:
                print(f"재료: {result['ingredient']}")
                print(result["error"])
            else:
                print(f"재료: {result['ingredient']}")
                print(f"url: {result['url']}")
                print(f"상품명: {result['product_name']}")
                print(f"가격: {result['product_price']}\n")
    else:
        print("추천된 레시피가 없습니다.")

# Main function to recommend recipes and search for ingredients
def main():
    # Load the model and embeddings
    loaded_model, loaded_embeddings = load_model_and_embeddings('model_embeddings.pkl')

    # Example user input
    user_input = input('뭘 도와줄까: ')
    model_name = 'bert-base-multilingual-cased'
    tokenizer = BertTokenizer.from_pretrained(model_name)

    # Get recommended recipes
    recommendations = recommend_recipe(user_input, loaded_model, loaded_embeddings, tokenizer)
    print(recommendations)

    # Process recommendations
    process_recommendations(recommendations)

# Run the main function
if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'model_embeddings.pkl'